A Simple Measure of Gene Order Similarityclick to collapse
In “Enumerating Gene Orders”, we started talking about comparing the order of genes on a chromosome taken from two different species and moved around by rearrangements throughout the course of evolution.

One very simple way of comparing genes from two chromosomes is to search for the largest collection of genes that are found in the same order in both chromosomes. To do so, we will need to apply our idea of permutations. Say that two chromosomes share n
 genes; if we label the genes of one chromosome by the numbers 1 through n in the order that they appear, then the second chromosome will be given by a permutation of these numbered genes. To find the largest number of genes appearing in the same order, we need only to find the largest collection of increasing elements in the permutation.

Problem
A subsequence of a permutation is a collection of elements of the permutation in the order that they appear. For example, (5, 3, 4) is a subsequence of (5, 1, 3, 4, 2).

A subsequence is increasing if the elements of the subsequence increase, and decreasing if the elements decrease. For example, given the permutation (8, 2, 1, 6, 5, 7, 4, 3, 9), an increasing subsequence is (2, 6, 7, 9), and a decreasing subsequence is (8, 6, 5, 4, 3). You may verify that these two subsequences are as long as possible.

Given: A positive integer n≤10000
 followed by a permutation π
 of length n
.

Return: A longest increasing subsequence of π
, followed by a longest decreasing subsequence of π
.

Sample Dataset

5

5 1 4 2 3


Sample Output

1 2 3

5 4 2

Citationclick to collapse
Adapted from Jones & Pevzner, *An Introduction to Bioinformatics Algorithms, Problem 6.48.

(Mine)
Rules for solution:
1. Sequence must be increasing
2. Sequence can skip base sequence members if they increase in future
3. Sequence can contain members from other sequences
    3a. I must consider all unique sequence members as their own node, which all increasing subsequences come off of
    3b. However, I must keep track of existing sequence members so that I can identify when a subsequence is increasing
4. Sequences may be of any length (but less than or equal to n by definition)
5. I must keep track of the longest increasing sequence thus far and replace it when a longer sequence is observed

Thoughts:

I could have an O(n^2) solution, but that would suck. I want an O(n) solution.
I need to traverse the sequence one, making sure to collect:
* All unique increasing subsequences

Tree - 
5 1 4 2 3
Nodes
1 2 3 4 5

Node 1
1-4
1-2-3
1-3

Node 2
2-3

Node 3
3

Node 4


In [40]:
from tqdm import tqdm_notebook as tqdm
def get_longest_subsequences(sequence):
    #build a tree
    increasing_nodes = {

    }
    decreasing_nodes = {

    }
    longest_increasing_sequence_length = 0
    longest_increasing_sequence = None
    longest_decreasing_sequence_length = 0
    longest_decreasing_sequence = None
    for char in tqdm(map(int,sequence)):
        #add char to nodes if not present
        if char not in increasing_nodes:
            increasing_nodes[char] = [[char]]
            if longest_increasing_sequence is None:
                longest_increasing_sequence = increasing_nodes[char]
                longest_increasing_sequence_length = 1
        if char not in decreasing_nodes:
            decreasing_nodes[char] = [[char]]
            if longest_decreasing_sequence is None:
                longest_decreasing_sequence = decreasing_nodes[char]
                longest_decreasing_sequence_length = 1
            
        #check all nodes for sequences that would be increasing if char was added
        new_increasing_nodes = increasing_nodes.copy()
        new_decreasing_nodes = decreasing_nodes.copy()
        for node in list(increasing_nodes.keys()):
            if char>node:
                for increasing_sequence_i in range(len(increasing_nodes[node])):
                    if char > increasing_nodes[node][increasing_sequence_i][-1]:
                        new_sequence = increasing_nodes[node][increasing_sequence_i] + [char]
                        new_increasing_nodes[node].append(new_sequence)
                        increasing_sequence_length = len(new_sequence)
                        if increasing_sequence_length > longest_increasing_sequence_length:
                            longest_increasing_sequence_length = increasing_sequence_length
                            longest_increasing_sequence = new_sequence
        increasing_nodes = new_increasing_nodes
        for node in list(decreasing_nodes.keys()):
            if char<node:
                for decreasing_sequence_i in range(len(decreasing_nodes[node])):
                    if char < decreasing_nodes[node][decreasing_sequence_i][-1]:
                        new_sequence = decreasing_nodes[node][decreasing_sequence_i] + [char]
                        new_decreasing_nodes[node].append(new_sequence)
                        decreasing_sequence_length = len(new_sequence)
                        if decreasing_sequence_length > longest_decreasing_sequence_length:
                            longest_decreasing_sequence_length = decreasing_sequence_length
                            longest_decreasing_sequence = new_sequence
                    
                    #TODO: Analyze why this is incorrect (the above was generated by Copilot Chat)
                    # if char>nodes[node][increasing_sequence_i][-1]:
                    #     new_nodes[node][increasing_sequence_i].append(char)
                    #     print(new_nodes)
                    #     new_nodes[node].append(nodes[node][increasing_sequence_i])
                    #     print(new_nodes)
                    #     increasing_sequence_length = len(new_nodes[node][increasing_sequence_i])
                    #     if increasing_sequence_length>longest_sequence_length:
                    #         longest_sequence_length = increasing_sequence_length
                    #         longest_sequence = new_nodes[node][increasing_sequence_i]
        decreasing_nodes = new_decreasing_nodes
        
    return longest_increasing_sequence, longest_decreasing_sequence


In [41]:
test_sequence = [5,1,4,2,3]
get_longest_subsequences(test_sequence)

/tmp/ipykernel_10161/144711621.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for char in tqdm(map(int,sequence)):


0it [00:00, ?it/s]

([1, 2, 3], [5, 4, 2])

In [42]:
sequence = open('rosalind_lgis.txt').read().split('\n')[1].split(' ')
print(len(sequence))
get_longest_subsequences(sequence)

9171


/tmp/ipykernel_10161/144711621.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for char in tqdm(map(int,sequence)):


0it [00:00, ?it/s]

In [30]:
#the above is slow as hell
#the below is a faster solution
def longest_increasing_subsequence(sequence):
    n = len(sequence)
    lis = [1] * n

    for i in range (1, n):
        for j in range(0, i):
            if sequence[i] > sequence[j] and lis[i] < lis[j] + 1:
                lis[i] = lis[j]+1

    maximum = max(lis)

    result = []
    for i in reversed(range(n)):
        if maximum == lis[i]:
            result.append(sequence[i])
            maximum -= 1

    return result[::-1]

def longest_decreasing_subsequence(sequence):
    return longest_increasing_subsequence(sequence[::-1])[::-1]

sequence = [5, 2, 8, 6, 3, 6, 9, 7]
lis = longest_increasing_subsequence(sequence)
lds = longest_decreasing_subsequence(sequence)

print("Longest Increasing Subsequence:", lis)
print("Longest Decreasing Subsequence:", lds)
sequence = [8,2,1,6,5,7,4,3,9]
lis = longest_increasing_subsequence(sequence)
lds = longest_decreasing_subsequence(sequence)
def get_longest_subsequences(sequence):
    return longest_increasing_subsequence(sequence), longest_decreasing_subsequence(sequence)
print("Longest Increasing Subsequence:", lis)
print("Longest Decreasing Subsequence:", lds)

Longest Increasing Subsequence: [2, 3, 6, 7]
Longest Decreasing Subsequence: [8, 6, 3]
Longest Increasing Subsequence: [1, 5, 7, 9]
Longest Decreasing Subsequence: [8, 6, 5, 4, 3]


In [31]:
def is_monotonic(sequence,reversed=False):
    if reversed:
        sequence = sequence[::-1]
    for i in range(len(sequence)-1):
        if sequence[i]>sequence[i+1]:
            return False
    return True

In [33]:
sequence = tuple(map(int,open('rosalind_lgis (4).txt').read().split('\n')[1].split(' ')))
#print(sequence)
#print(len(sequence))
longest_subsequences = get_longest_subsequences(sequence)
assert is_monotonic(longest_subsequences[0]) and is_monotonic(longest_subsequences[1],reversed=True)
with open('ls.txt','w') as results:
    results.write('\n'.join([' '.join(map(str,s)) for s in get_longest_subsequences(sequence)]))

In [ ]:
#finally correct...